# DC-GAN

**Paper: Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks**

**Link: https://arxiv.org/abs/1511.06434**

**Authors: Alec Radford, Luke Metz, Soumith Chintala**

## Abstract
In recent years, supervised learning with convolutional networks (CNNs) has seen huge adoption in computer vision applications. Comparatively, unsupervised learning with CNNs has received less attention. In this work we hope to help bridge the gap between the success of CNNs for supervised learning and unsupervised learning. We introduce a class of CNNs called deep convolutional generative adversarial networks (DCGANs), that have certain architectural constraints, and demonstrate that they are a strong candidate for unsupervised learning. Training on various image datasets, we show convincing evidence that our deep convolutional adversarial pair learns a hierarchy of representations from object parts to scenes in both the generator and discriminator. Additionally, we use the learned features for novel tasks - demonstrating their applicability as general image representations.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
    
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.optim import Adam

from tqdm.notebook import tqdm

from glob import glob
import matplotlib.pyplot as plt


import pytorch_lightning as pl


torch.manual_seed(0)

In [2]:
def show_tensor_images(image_tensor, num_images=25, size=(3, 64, 64)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [3]:
# Configurations are from DCGAN paper
z_dim = 100
batch_size = 128
lr = 0.0002

beta_1 = 0.5 
beta_2 = 0.999
device = 'cpu' # 'cuda'

## Create dataloader

In [4]:
from PIL import Image

In [17]:
path = '/Users/aniket/data/tiny-imagenet-200/train'

class ImageDataset(Dataset):
    def __init__(self, path, transform=None):
        self.transform = transform
        self.files = glob(f'{path}/*/images/*.JPEG')
    
    def __len__(self): return len(self.files)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        label = file.split('/')[-3]
        image = Image.open(file).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [18]:
# You can tranform the image values to be between -1 and 1 (the range of the tanh activation)
transform=transforms.Compose([
                            transforms.Resize(64),
                            transforms.CenterCrop(64),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                        ])


ds = ImageDataset(path, transform)
dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

# Network

GANs are made up of two entities - Generator and Discriminator.

## Generator
Generator learns the data distribution and creates fake images.

In [7]:
class Generator(nn.Module):
    
    def __init__(self, in_channels=3, z_dim=100):
        super(Generator, self).__init__()
        
        self.gen = nn.Sequential(
            self.create_upblock(z_dim, 1024, kernel_size=4, stride=1, padding=0),
            self.create_upblock(1024, 512, kernel_size=4, stride=2, padding=1),
            self.create_upblock(512, 256, kernel_size=4, stride=2, padding=1),
            self.create_upblock(256, 128, kernel_size=4, stride=2, padding=1),
            self.create_upblock(128, 3, kernel_size=4, stride=2, padding=1, final_layer=True),
        )
    
    def create_upblock(self, in_channels, out_channels, kernel_size=5, stride=2, padding=1, final_layer=False):
        if final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.Tanh()
            )
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
    
    def forward(self, noise):
        """
        noise: random vector of shape=(N, 100, 1, 1)
        """
        assert len(noise.shape)==4, 'random vector of shape=(N, 100, 1, 1)'
        
        return self.gen(noise)
def get_noise(bs, z_dim):
    noise = torch.randn(bs, z_dim, 1, 1, device=device)
    return noise

# Discriminator

In [8]:
class Discriminator(nn.Module):

    def __init__(self, im_chan=3, hidden_dim=32):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            self.make_disc_block(im_chan, hidden_dim),
            self.make_disc_block(hidden_dim, hidden_dim * 2),
            self.make_disc_block(hidden_dim*2, hidden_dim * 4, stride=1),
            self.make_disc_block(hidden_dim*4, hidden_dim * 4, stride=2),
            self.make_disc_block(hidden_dim * 4, 1, final_layer=True),
        )

    def make_disc_block(self, input_channels, output_channels, kernel_size=4, stride=2, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(0.2)
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride)
            )

    def forward(self, image):
        disc_pred = self.disc(image)
        return disc_pred.view(len(disc_pred), -1)

Initialise Generator and Discriminator

In [9]:
criterion = nn.BCEWithLogitsLoss()

In [37]:
class GAN(pl.LightningModule):

    def __init__(self, learning_rate, in_channels=3, hidden_dim=32, z_dim=100, **kwargs):
        super().__init__()
        
        self.z_dim = z_dim
        self.learning_rate=learning_rate
        
        self.gen = Generator(in_channels, z_dim=z_dim).to(device)
        self.disc = Discriminator(im_chan=in_channels, hidden_dim=hidden_dim).to(device)
        
        self.save_hyperparameters()

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        return self.generator(x)

    def disc_step(self, x):
        """
        x: real image
        """
        # generate fake images
        noise = get_noise(batch_size, self.z_dim)
        fake_images = self.gen(noise).detach()

        # get discriminator outputs
        real_logits = self.disc(x)
        fake_logits = self.disc(x)
        
        # real loss
        real_loss = criterion(real_logits, torch.ones_like(real_logits))

        # fake loss
        fake_loss = criterion(fake_logits, torch.zeros_like(fake_logits))

        disc_loss = (fake_loss + real_loss) / 2

        return self.log('disc_loss', disc_loss, on_epoch=True, prog_bar=True)

    
    def gen_step(self, x):
        # generate fake images
        noise = get_noise(batch_size, self.z_dim)
        fake_images = self.gen(noise)

        fake_logits = self.disc(fake_images)
        fake_loss = criterion(fake_logits, torch.ones_like(fake_logits))

        gen_loss = fake_loss
        return self.log('gen_loss', gen_loss, on_epoch=True, prog_bar=True)


    def training_step(self, batch, batch_idx, optimizer_idx):
        x, _ = batch
        
        loss = None
        # first get gen loss [opt_g, opt_d]
        if optimizer_idx == 0:
            loss = self.gen_step(x)
        
        if optimizer_idx == 1:
            loss = self.disc_step(x)
        
        return loss


    def configure_optimizers(self):
        lr = self.hparams.learning_rate

        opt_g = torch.optim.Adam(self.gen.parameters(), lr=lr)
        opt_d = torch.optim.Adam(self.disc.parameters(), lr=lr)
        return [opt_g, opt_d], []

In [38]:
model = GAN(learning_rate=lr, z_dim=z_dim)

In [39]:
trainer = pl.Trainer()

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [40]:
trainer.fit(model, dl)


  | Name | Type          | Params
---------------------------------------
0 | gen  | Generator     | 12.7 M
1 | disc | Discriminator | 430 K 
---------------------------------------
13.1 M    Trainable params
0         Non-trainable params
13.1 M    Total params
/Users/aniket/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/Users/aniket/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: training_step returned None if it was on purpose, ignore this warning...
  warnings.warn(*args, **kwargs)


/Users/aniket/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1